Saves an email attachment to a specific folder mapped from SharePoint. Transfers the information from individually scanned files to a master log, then overwrites those files with blank templates.

In [1]:
# save full inventory attachment to sharepoint
# copy the content from all of the individual files into the changelog file, "outsytd.xlsx"
# copy template files over the previously populated ones
# delete all emails and attachments

import win32com.client
import glob
import pandas as pd
from datetime import date
from functools import reduce
import shutil
import re

#copy over the latest attachment/report from Eagle for the entire inventory Store 7
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Select main Inbox
inbox = outlook.GetDefaultFolder(6).Folders.Item('Epicor Reports')
messages = inbox.Items

# sender = 'reports@website.com'
subject = 'Epicor Compass Query SD Full Inventory Store 7'
path = r'C:\Users\USER\OneDrive - COMPANY\Documents - Inventory\Outs\Templates\\'

try:
    for message in messages:
        try:
            s = message.sender
            s = str(s)
            ss = message.subject
            ss = str(ss)
            # if s == sender & ss == subject:
            if ss == subject:
                attachments = message.Attachments
                # return the first item in attachments
                attachment = attachments.Item(1)
                # the name of attachment file      
                attachment_name = str(attachment).lower()
                attachment.SaveAsFile(path + '\\' + attachment_name)
        except:
            print('Attachment NOT Saved')
            pass
except:
    pass

#loop through all of the scanned files and copy over the content into OutsYTD.xlsx

fmask = 'C:/Users/USER/OneDrive - COMPANY/Documents - Inventory/Outs/Files/*.xlsx'

# lst of column names which needs to be string
lst_str_cols = ['UPC']
# use dictionary comprehension to make dict of dtypes
dict_dtypes = {x : 'str'  for x in lst_str_cols}
# use dict on dtypes

dfs = []
for f in glob.glob(fmask):
    df = pd.read_excel(f, dtype=dict_dtypes)
    dfs.append(df)

# concatenated
dfOuts = pd.concat(dfs, ignore_index=True)

dfOutsYTD = pd.read_excel(r'C:\Users\USER\OneDrive - COMPANY\Documents - Inventory\Outs\Output\OutsYTD.xlsx', dtype=dict_dtypes)

dfOutsYTD = pd.concat([dfOutsYTD, dfOuts]).fillna(date.today())

dfOutsYTD['UPC'] = pd.DataFrame(dfOutsYTD['UPC'], dtype=object)
# dfOutsYTD['Date'] = pd.DataFrame(dfOutsYTD['Date'], dtype=object)

dfOutsYTD['Date'] = dfOutsYTD['Date'].astype(str).str.rstrip(' 00:00:00')

# lst of column names which needs to be string
lst_str_cols = ['Item Number', 'Department Code', 'UPC']
# use dictionary comprehension to make dict of dtypes
dict_dtypes = {x : 'str'  for x in lst_str_cols}
# use dict on dtypes

dfFullInventory = pd.read_csv(r'C:\Users\USER\OneDrive - COMPANY\Documents - Inventory\Outs\Templates\FullInventory.csv', dtype=dict_dtypes)

data_frames = [dfOutsYTD, dfFullInventory]

df_merged = reduce(lambda left,right: pd.merge(left,right,left_on=['UPC'],right_on=['UPC Code'], how='left'), data_frames).fillna('null')

# dropping ALL duplicate values
df_merged.drop_duplicates(subset = ['UPC', 'Date', 'Location'], keep = False, inplace = True)

df_merged.drop(df_merged.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]], axis = 1, inplace=True)

# Create a Pandas Excel writer using XlsxWriter as the engine.

with pd.ExcelWriter(r'C:\Users\USER\OneDrive - COMPANY\Documents - Inventory\Outs\Output\OutsYTD.xlsx', engine='xlsxwriter') as writer:

    # Convert the dataframe to an XlsxWriter Excel object.
    dfOutsYTD.to_excel(writer, sheet_name='Sheet1', index=False, header=True)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # Add some cell formats.
    format1 = workbook.add_format({'num_format': '@'})

    # Note: It isn't possible to format any cells that already have a format such
    # as the index or headers or any cells that contain dates or datetimes.

    # Set the column width and format.
    worksheet.set_column('A:B', None, format1)

#loop through all of the scanned files and copy over the templates for them

fmask = 'C:/Users/USER/OneDrive - COMPANY/Documents - Inventory/Outs/Files/*.xlsx'

dfs = []
for f in glob.glob(fmask):
    f = f.replace(r'C:/Users/USER/OneDrive - COMPANY/Documents - Inventory/Outs/Files\\', '')
    original = r'C:\Users\USER\OneDrive - COMPANY\Documents - Inventory\Outs\Templates\\' + f
    target = r'C:\Users\USER\OneDrive - COMPANY\Documents - Inventory\Outs\Files\\' + f
    shutil.copyfile(original, target)
    
#copy over the latest attachment/report from Eagle for the entire inventory Store 7
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Select main Inbox
inbox = outlook.GetDefaultFolder(6).Folders.Item('Reports')
messages = inbox.Items

# Delete all messages from a specific sender
sender = 'reports@website.com'
try:
    for message in messages:
        try:
            s = message.sender
            s = str(s)
            if s == sender:
                 message.Delete()
        except:
            pass
except:
    pass
    
print('Done.')

Done.


In [3]:
import win32com.client

#copy over the latest attachment/report from Eagle for the entire inventory Store 7
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Select main Inbox
inbox = outlook.GetDefaultFolder(6).Folders.Item('Reports')
messages = inbox.Items

# Delete all messages from a specific sender
sender = 'reports@website.com'
try:
    for message in messages:
        try:
            s = message.sender
            s = str(s)
            if s == sender:
                 message.Delete()
        except:
            pass
except:
    pass
    
print('Deleted.')

Deleted.


In [2]:
len(df_merged)

703

In [9]:
myColumnCount = len(df_merged.columns)

i = 0

while i < myColumnCount:
    col = df_merged.columns[i]
    # Get column index position of column
    col_index = list(df_merged.columns).index(df_merged.columns[i])
    print(str(col_index) + ' - ' + col)
    i += 1

0 - UPC
1 - Date
2 - Item Number
3 - Item Description
4 - Department Code
5 - Department Name
6 - Class Code
7 - Class Name
8 - Fineline Code
9 - Fineline Name
10 - Location
11 - Location 2
12 - Location 3
13 - Location 4
14 - Location 5
15 - Location 6
16 - UPC Code
17 - Store Closeout?
18 - Quantity on Hand
19 - Quantity on Order
